In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the data into a pandas dataframe
df = pd.read_csv('flights.csv')


In [2]:
df.head(5)

,Destination,DepartureDate,ReturnDate,Airline,FlightTime,Locations,TotalFlightTime,Layover,Price,PriceType,Details
0,"New York, NY (NYC-All Airports)",Apr 18,Apr 19,Delta,11:29am - 4:04pm,Baltimore (BWI) - New York (LGA),4h 35m (1 stop),1h 20m in Detroit (DTW),$241,Roundtrip per traveler,Carry-on included
1,"New York, NY (NYC-All Airports)",Apr 18,Apr 19,Delta,10:43am - 3:45pm,Baltimore (BWI) - New York (LGA),5h 2m (1 stop),56m in Atlanta (ATL),$241,Roundtrip per traveler,Carry-on included
2,"New York, NY (NYC-All Airports)",Apr 18,Apr 19,Delta,8:00am - 1:17pm,Baltimore (BWI) - New York (LGA),5h 17m (1 stop),1h 4m in Atlanta (ATL),$241,Roundtrip per traveler,Carry-on included
3,"New York, NY (NYC-All Airports)",Apr 18,Apr 19,Delta,2:45pm - 8:09pm,Baltimore (BWI) - New York (LGA),5h 24m (1 stop),1h 6m in Atlanta (ATL),$241,Roundtrip per traveler,Carry-on included
4,"New York, NY (NYC-All Airports)",Apr 18,Apr 19,Delta,6:00pm - 11:44pm,Baltimore (BWI) - New York (LGA),5h 44m (1 stop),1h 29m in Atlanta (ATL),$241,Roundtrip per traveler,Carry-on included


In [5]:
# Define a regular expression pattern to extract "Source"
pattern_source = r'^([^\s]+)'

# Apply the regular expression pattern to the 'locations' column
df['Source'] = df['Locations'].str.extract(pattern_source)

In [6]:
# Define a function to extract the city name from a location string
def extract_destinationcity(location_string):
    city = location_string.split(',')[0]
    return city

# Apply the function to the 'Locations' column to extract the city names and store them in a new 'City' column
df['Destination'] = df['Destination'].apply(extract_destinationcity)

In [7]:
# Define regular expression pattern to extract hours and minutes
pattern = r'(\d+)h (\d+)m'

# Extract hours and minutes using regular expressions and create new columns
df[['TotalTime_Hours', 'TotalTime_Minutes']] = df['TotalFlightTime'].str.extract(pattern)
df[['Layover_Hours', 'Layover_Minutes']] = df['Layover'].str.extract(pattern)

# Convert hours and minutes columns to numeric type
df['TotalTime_Hours'] = pd.to_numeric(df['TotalTime_Hours'])
df['TotalTime_Minutes'] = pd.to_numeric(df['TotalTime_Minutes'])
df['Layover_Hours'] = pd.to_numeric(df['Layover_Hours'])
df['Layover_Minutes'] = pd.to_numeric(df['Layover_Minutes'])


In [8]:
df['TotalTime_in_mins'] = (df['TotalTime_Hours'] * 60) + (df['TotalTime_Minutes'])
df['Layover_in_mins'] = (df['Layover_Hours'] * 60) + (df['Layover_Minutes'])

In [9]:
# replace NaN values with 0 in the dataframe
df.fillna(0, inplace=True)

# convert the float column to int
df['Layover_in_mins'] = df['Layover_in_mins'].round().astype(int)

# Calculate the duration of each flight
df['Duration_in_mins'] = df['TotalTime_in_mins'] - df['Layover_in_mins']

# remove the currency sign from the price column and convert to numeric type
df['Price'] = pd.to_numeric(df['Price'].str.replace(r'[^0-9.]', ''))

# Remove unwanted columns
df.drop(['TotalTime_Hours', 'TotalTime_Minutes', 'Layover_Hours', 'Layover_Minutes','Locations','Details','PriceType','TotalTime_in_mins','Layover_in_mins'], axis=1, inplace=True)

<ipython-input-9-3dd2cdd06349>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Price'] = pd.to_numeric(df['Price'].str.replace(r'[^0-9.]', ''))


In [14]:
df.head(10)

,Destination,DepartureDate,ReturnDate,Airline,FlightTime,TotalFlightTime,Layover,Price,Source,Duration_in_mins
0,New York,Apr 18,Apr 19,Delta,11:29am - 4:04pm,4h 35m (1 stop),1h 20m in Detroit (DTW),241,Baltimore,195
1,New York,Apr 18,Apr 19,Delta,10:43am - 3:45pm,5h 2m (1 stop),56m in Atlanta (ATL),241,Baltimore,302
2,New York,Apr 18,Apr 19,Delta,8:00am - 1:17pm,5h 17m (1 stop),1h 4m in Atlanta (ATL),241,Baltimore,253
3,New York,Apr 18,Apr 19,Delta,2:45pm - 8:09pm,5h 24m (1 stop),1h 6m in Atlanta (ATL),241,Baltimore,258
4,New York,Apr 18,Apr 19,Delta,6:00pm - 11:44pm,5h 44m (1 stop),1h 29m in Atlanta (ATL),241,Baltimore,255
5,New York,Apr 18,Apr 19,Delta,5:50am - 11:59am,6h 9m (1 stop),2h 48m in Detroit (DTW),241,Baltimore,201
6,New York,Apr 18,Apr 19,Spirit Airlines,6:22am - 2:58pm,8h 36m (1 stop),4h 32m in Atlanta (ATL),248,Baltimore,244
7,New York,Apr 18,Apr 19,American Airlines,10:15am - 3:25pm,5h 10m (1 stop),1h 47m in Charlotte (CLT),263,Baltimore,203
8,New York,Apr 18,Apr 19,Delta,6:00am - 10:49am,4h 49m (1 stop),38m in Atlanta (ATL),289,Baltimore,289
9,New York,Apr 18,Apr 19,Delta,6:00am - 11:17am,5h 17m (1 stop),1h 23m in Atlanta (ATL),289,Baltimore,234


In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# create a new column 'features' that combines the relevant columns
features = ['Destination', 'Airline', 'Source']
df['features'] = df[features].apply(lambda x: ' '.join(x), axis=1)

# create a TF-IDF vectorizer object to convert the features column into a matrix of feature vectors
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['features'])

# compute the cosine similarity matrix between all flights
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# define a function to get the top n most similar flights to a given flight
def get_top_similar_flights(flight_id, n=10):
    # compute the pairwise cosine similarities between the given flight and all other flights
    sim_scores = list(enumerate(cosine_sim[flight_id]))
    # sort the flights by their similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # get the top n most similar flights (excluding the given flight itself)
    top_similar_flights = [i[0] for i in sim_scores[1:n+1]]
    return top_similar_flights

# define a function to recommend the cheapest flights based on a given flight
def recommend_cheapest_flights(flight_id):
    # get the top 10 most similar flights to the given flight
    top_similar_flights = get_top_similar_flights(flight_id)
    # sort the similar flights by price in ascending order
    sorted_similar_flights = df.iloc[top_similar_flights].sort_values(by=['Price','Duration_in_mins'], ascending=True)
    # return the top 10 cheapest flights
    return sorted_similar_flights.head(10)

# example usage: recommend the cheapest flights similar to flight with index 100
recommendations = recommend_cheapest_flights(5)


In [31]:
recommendations.head()

,Destination,DepartureDate,ReturnDate,Airline,FlightTime,TotalFlightTime,Layover,Price,Source,Duration_in_mins,features
5,New York,Apr 18,Apr 19,Delta,5:50am - 11:59am,6h 9m (1 stop),2h 48m in Detroit (DTW),241,Baltimore,201,New York Delta Baltimore
2,New York,Apr 18,Apr 19,Delta,8:00am - 1:17pm,5h 17m (1 stop),1h 4m in Atlanta (ATL),241,Baltimore,253,New York Delta Baltimore
4,New York,Apr 18,Apr 19,Delta,6:00pm - 11:44pm,5h 44m (1 stop),1h 29m in Atlanta (ATL),241,Baltimore,255,New York Delta Baltimore
3,New York,Apr 18,Apr 19,Delta,2:45pm - 8:09pm,5h 24m (1 stop),1h 6m in Atlanta (ATL),241,Baltimore,258,New York Delta Baltimore
1,New York,Apr 18,Apr 19,Delta,10:43am - 3:45pm,5h 2m (1 stop),56m in Atlanta (ATL),241,Baltimore,302,New York Delta Baltimore
